In [36]:
import altair as alt
import pandas as pd
import numpy as np

# ---------- A) Cargar CSV (robusto) ----------
fname = "pistacholargo.csv"   # cambia si tu archivo tiene otro nombre o ruta
try:
    df = pd.read_csv(fname, sep=';')   # tu CSV usa ; como separador
except Exception as e:
    raise RuntimeError(f"Error leyendo {fname}: {e}")

# ---------- B) Inspección rápida (imprime para debug) ----------
print("Columnas originales:", df.columns.tolist())
print("Primeras filas (raw):")
print(df.head().to_string(index=False))

# ---------- C) Normalizar nombres de columnas ----------
df.columns = df.columns.str.strip().str.lower().str.replace('ñ', 'n')
print("Columnas normalizadas:", df.columns.tolist())

# ---------- D) Validaciones básicas ----------
required = ['especie']
for r in required:
    if r not in df.columns:
        raise KeyError(f"No se encontró la columna obligatoria '{r}'. Columnas disponibles: {df.columns.tolist()}")

# Detectar columna del año (variantes posibles)
year_col = None
for c in df.columns:
    if 'ano' in c or 'año' in c or 'year' in c or c.strip() == 'a':
        year_col = c
        break
if year_col is None:
    raise KeyError(f"No pude detectar la columna del año. Columnas: {df.columns.tolist()}")
print("Columna detectada como año:", year_col)

# Detectar columna de hectareas (variantes)
hect_col = None
for c in df.columns:
    if 'hect' in c or 'ha' == c or 'hectareas' in c:
        hect_col = c
        break
if hect_col is None:
    # Si no la detecta, intenta usar la segunda columna distinta a especie/año
    candidates = [c for c in df.columns if c not in [year_col, 'especie']]
    if candidates:
        hect_col = candidates[0]
    else:
        raise KeyError(f"No pude detectar la columna de hectareas. Columnas: {df.columns.tolist()}")
print("Columna detectada como hectareas:", hect_col)

# ---------- E) Normalizar tipos y renombrar a campos esperados ----------
df = df.rename(columns={year_col: 'ano', hect_col: 'hectareas'})

# Quitar filas vacías en las columnas clave
df = df.dropna(subset=['especie', 'ano', 'hectareas'])

# Convertir tipos (con manejo de errores)
df['ano'] = pd.to_numeric(df['ano'], errors='coerce').astype('Int64')
df['hectareas'] = pd.to_numeric(df['hectareas'], errors='coerce')

# Eliminar filas con valores no convertibles
bad = df[df['ano'].isna() | df['hectareas'].isna()]
if not bad.empty:
    print("Advertencia: se eliminarán filas con año/hectareas inválidos:")
    print(bad.to_string(index=False))
    df = df.dropna(subset=['ano', 'hectareas'])

# Forzar ano a int (normalizamos a Python int para Altair)
df['ano'] = df['ano'].astype(int)
df['hectareas'] = df['hectareas'].astype(float)

print("Primeras filas (limpias):")
print(df.head().to_string(index=False))

# ---------- F) Crear ID por especie (orden estable) ----------
especies = sorted(df['especie'].unique())
print(f"Número de especies detectadas: {len(especies)}")
id_map = {esp: i for i, esp in enumerate(especies)}
df['id'] = df['especie'].map(id_map)

# ---------- G) Crear tabla de locations reproducible ----------
np.random.seed(0)
n = len(especies)
# Distribuimos las posiciones en una rejilla simple para que no se monten
cols = int(np.ceil(np.sqrt(n)))
rows = int(np.ceil(n / cols))
xs = np.linspace(-2, 2, cols)
ys = np.linspace(-2, 2, rows)
positions = []
for r in range(rows):
    for c in range(cols):
        positions.append((xs[c], ys[r]))
positions = positions[:n]

locations = pd.DataFrame({
    'id': list(id_map.values()),
    'especie': list(id_map.keys()),
    'x': [p[0] for p in positions],
    'y': [p[1] for p in positions]
})

# ---------- H) Merge datos + metadata ----------
data = pd.merge(df, locations, on=['id', 'especie'], how='left')

# Comprobar
print("Ejemplo de data mergeada:")
print(data[['especie','id','ano','hectareas','x','y']].head().to_string(index=False))

# ---------- I) Renombrar para parecerse al código original (time, value) ----------
data = data.rename(columns={'ano': 'time', 'hectareas': 'value'})

# ---------- J) Crear selector y gráficos ----------
selector = alt.selection_point(fields=['id'])

# color condicional: seleccionado por id, si no seleccionado gris
color = alt.condition(
    selector,
    alt.Color('id:O', legend=None),
    alt.value('lightgray')
)

base = alt.Chart(data).properties(width=300, height=300).add_params(selector)

points = base.mark_point(filled=True, size=200).encode(
    x='mean(x):Q',
    y='mean(y):Q',
    color=color,
    tooltip=['especie:N', 'id:Q']
).properties(
    title='Especies (haz clic para seleccionar)'
)

line = alt.Chart(data).mark_line(point=True).encode(
    x=alt.X('time:O', title='Año'),
    y=alt.Y('value:Q', title='Hectáreas', scale=alt.Scale(zero=False)),
    color=alt.Color('id:O', legend=None),
    tooltip=['especie:N','time:O','value:Q']
).transform_filter(
    selector
).properties(
    width=500,
    height=300,
    title='Serie temporal de la especie seleccionada'
)

# Mostrar lado a lado
chart = (points | line).configure_title(fontSize=14, anchor='start')
chart


Columnas originales: ['especie', 'año', 'hectareas', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26']
Primeras filas (raw):
 especie    año  hectareas  Unnamed: 3  Unnamed: 4  Unnamed: 5  Unnamed: 6  Unnamed: 7  Unnamed: 8  Unnamed: 9  Unnamed: 10  Unnamed: 11  Unnamed: 12  Unnamed: 13  Unnamed: 14  Unnamed: 15  Unnamed: 16  Unnamed: 17  Unnamed: 18  Unnamed: 19  Unnamed: 20  Unnamed: 21  Unnamed: 22  Unnamed: 23  Unnamed: 24  Unnamed: 25  Unnamed: 26
Almendro 2024.0    8324.11         NaN         NaN         NaN         NaN         NaN         NaN         NaN          NaN          NaN          NaN          NaN          NaN          NaN          NaN          NaN          NaN          NaN

alt.HConcatChart(...)

In [41]:
import altair as alt
import pandas as pd
import numpy as np

# ---------- A) Cargar CSV (robusto) ----------
fname = "pistacholargo.csv"   # cambia si tu archivo tiene otro nombre o ruta
try:
    df = pd.read_csv(fname, sep=';')   # tu CSV usa ; como separador
except Exception as e:
    raise RuntimeError(f"Error leyendo {fname}: {e}")

# ---------- B) Inspección rápida (imprime para debug) ----------
print("Columnas originales:", df.columns.tolist())
print("Primeras filas (raw):")
print(df.head().to_string(index=False))

# ---------- C) Normalizar nombres de columnas ----------
df.columns = df.columns.str.strip().str.lower().str.replace('ñ', 'n')
print("Columnas normalizadas:", df.columns.tolist())

# ---------- D) Validaciones básicas ----------
required = ['especie']
for r in required:
    if r not in df.columns:
        raise KeyError(f"No se encontró la columna obligatoria '{r}'. Columnas disponibles: {df.columns.tolist()}")

# Detectar columna del año (variantes posibles)
year_col = None
for c in df.columns:
    if 'ano' in c or 'año' in c or 'year' in c or c.strip() == 'a':
        year_col = c
        break
if year_col is None:
    raise KeyError(f"No pude detectar la columna del año. Columnas: {df.columns.tolist()}")
print("Columna detectada como año:", year_col)

# Detectar columna de hectareas (variantes)
hect_col = None
for c in df.columns:
    if 'hect' in c or 'ha' == c or 'hectareas' in c:
        hect_col = c
        break
if hect_col is None:
    # Si no la detecta, intenta usar la segunda columna distinta a especie/año
    candidates = [c for c in df.columns if c not in [year_col, 'especie']]
    if candidates:
        hect_col = candidates[0]
    else:
        raise KeyError(f"No pude detectar la columna de hectareas. Columnas: {df.columns.tolist()}")
print("Columna detectada como hectareas:", hect_col)

# ---------- E) Normalizar tipos y renombrar a campos esperados ----------
df = df.rename(columns={year_col: 'ano', hect_col: 'hectareas'})

# Quitar filas vacías en las columnas clave
df = df.dropna(subset=['especie', 'ano', 'hectareas'])

# Convertir tipos (con manejo de errores)
df['ano'] = pd.to_numeric(df['ano'], errors='coerce').astype('Int64')
df['hectareas'] = pd.to_numeric(df['hectareas'], errors='coerce')

# Eliminar filas con valores no convertibles
bad = df[df['ano'].isna() | df['hectareas'].isna()]
if not bad.empty:
    print("Advertencia: se eliminarán filas con año/hectareas inválidos:")
    print(bad.to_string(index=False))
    df = df.dropna(subset=['ano', 'hectareas'])

# Forzar ano a int (normalizamos a Python int para Altair)
df['ano'] = df['ano'].astype(int)
df['hectareas'] = df['hectareas'].astype(float)

print("Primeras filas (limpias):")
print(df.head().to_string(index=False))

# ---------- F) Crear ID por especie (orden estable) ----------
especies = sorted(df['especie'].unique())
print(f"Número de especies detectadas: {len(especies)}")
id_map = {esp: i for i, esp in enumerate(especies)}
df['id'] = df['especie'].map(id_map)

# ---------- G) Preparar datos para el gráfico de burbujas ----------
# Crear posiciones para las especies (en línea horizontal)
especies_ordenadas = sorted(df['especie'].unique())
posiciones_x = {esp: i for i, esp in enumerate(especies_ordenadas)}
df['x_pos'] = df['especie'].map(posiciones_x)

# ---------- H) Renombrar para consistencia ----------
data = df.rename(columns={'ano': 'time', 'hectareas': 'value'})

# ---------- I) Crear gráficos ----------
# Selector de especie
species_selector = alt.selection_point(fields=['id'])

# Color condicional
color = alt.condition(
    species_selector,
    alt.Color('id:O', legend=None),
    alt.value('lightgray')
)

# Gráfico de burbujas - una fila por año
bubbles = alt.Chart(data).mark_circle(filled=True, opacity=0.8).encode(
    x=alt.X('x_pos:O', 
            axis=alt.Axis(title='Especies', labels=False, ticks=False),
            scale=alt.Scale(padding=0.5)),
    y=alt.Y('time:O', title='Año'),
    size=alt.Size('value:Q',
                 title='Hectáreas',
                 scale=alt.Scale(range=[50, 1500]),  # Ajustar según tus datos
                 legend=alt.Legend(orient='bottom')),
    color=color,
    tooltip=[
        'especie:N', 
        'time:O',
        alt.Tooltip('value:Q', title='Hectáreas', format=',.0f')
    ]
).properties(
    width=400,
    height=300,
    title='Hectáreas por Especie y Año'
).add_params(species_selector)

# Gráfico de series temporales
line = alt.Chart(data).mark_line(point=True).encode(
    x=alt.X('time:O', title='Año'),
    y=alt.Y('value:Q', title='Hectáreas', scale=alt.Scale(zero=False)),
    color=alt.Color('id:O', legend=None),
    tooltip=['especie:N','time:O','value:Q']
).transform_filter(
    species_selector
).properties(
    width=500,
    height=300,
    title='Serie Temporal de la Especie Seleccionada'
)

# Mostrar los gráficos
chart = (bubbles | line).configure_title(fontSize=14, anchor='start')

chart

Columnas originales: ['especie', 'año', 'hectareas', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26']
Primeras filas (raw):
 especie    año  hectareas  Unnamed: 3  Unnamed: 4  Unnamed: 5  Unnamed: 6  Unnamed: 7  Unnamed: 8  Unnamed: 9  Unnamed: 10  Unnamed: 11  Unnamed: 12  Unnamed: 13  Unnamed: 14  Unnamed: 15  Unnamed: 16  Unnamed: 17  Unnamed: 18  Unnamed: 19  Unnamed: 20  Unnamed: 21  Unnamed: 22  Unnamed: 23  Unnamed: 24  Unnamed: 25  Unnamed: 26
Almendro 2024.0    8324.11         NaN         NaN         NaN         NaN         NaN         NaN         NaN          NaN          NaN          NaN          NaN          NaN          NaN          NaN          NaN          NaN          NaN

alt.HConcatChart(...)

In [42]:
import altair as alt
import pandas as pd
import numpy as np

# ---------- A) Cargar CSV (robusto) ----------
fname = "pistacholargo.csv"   # cambia si tu archivo tiene otro nombre o ruta
try:
    df = pd.read_csv(fname, sep=';')   # tu CSV usa ; como separador
except Exception as e:
    raise RuntimeError(f"Error leyendo {fname}: {e}")

# ---------- B) Inspección rápida (imprime para debug) ----------
print("Columnas originales:", df.columns.tolist())
print("Primeras filas (raw):")
print(df.head().to_string(index=False))

# ---------- C) Normalizar nombres de columnas ----------
df.columns = df.columns.str.strip().str.lower().str.replace('ñ', 'n')
print("Columnas normalizadas:", df.columns.tolist())

# ---------- D) Validaciones básicas ----------
required = ['especie']
for r in required:
    if r not in df.columns:
        raise KeyError(f"No se encontró la columna obligatoria '{r}'. Columnas disponibles: {df.columns.tolist()}")

# Detectar columna del año (variantes posibles)
year_col = None
for c in df.columns:
    if 'ano' in c or 'año' in c or 'year' in c or c.strip() == 'a':
        year_col = c
        break
if year_col is None:
    raise KeyError(f"No pude detectar la columna del año. Columnas: {df.columns.tolist()}")
print("Columna detectada como año:", year_col)

# Detectar columna de hectareas (variantes)
hect_col = None
for c in df.columns:
    if 'hect' in c or 'ha' == c or 'hectareas' in c:
        hect_col = c
        break
if hect_col is None:
    # Si no la detecta, intenta usar la segunda columna distinta a especie/año
    candidates = [c for c in df.columns if c not in [year_col, 'especie']]
    if candidates:
        hect_col = candidates[0]
    else:
        raise KeyError(f"No pude detectar la columna de hectareas. Columnas: {df.columns.tolist()}")
print("Columna detectada como hectareas:", hect_col)

# ---------- E) Normalizar tipos y renombrar a campos esperados ----------
df = df.rename(columns={year_col: 'ano', hect_col: 'hectareas'})

# Quitar filas vacías en las columnas clave
df = df.dropna(subset=['especie', 'ano', 'hectareas'])

# Convertir tipos (con manejo de errores)
df['ano'] = pd.to_numeric(df['ano'], errors='coerce').astype('Int64')
df['hectareas'] = pd.to_numeric(df['hectareas'], errors='coerce')

# Eliminar filas con valores no convertibles
bad = df[df['ano'].isna() | df['hectareas'].isna()]
if not bad.empty:
    print("Advertencia: se eliminarán filas con año/hectareas inválidos:")
    print(bad.to_string(index=False))
    df = df.dropna(subset=['ano', 'hectareas'])

# Forzar ano a int (normalizamos a Python int para Altair)
df['ano'] = df['ano'].astype(int)
df['hectareas'] = df['hectareas'].astype(float)

print("Primeras filas (limpias):")
print(df.head().to_string(index=False))

# ---------- F) Crear ID por especie (orden estable) ----------
especies = sorted(df['especie'].unique())
print(f"Número de especies detectadas: {len(especies)}")
id_map = {esp: i for i, esp in enumerate(especies)}
df['id'] = df['especie'].map(id_map)

# ---------- G) Preparar datos para el gráfico de burbujas ----------
# Crear posiciones para las especies (en línea horizontal)
especies_ordenadas = sorted(df['especie'].unique())
posiciones_x = {esp: i for i, esp in enumerate(especies_ordenadas)}
df['x_pos'] = df['especie'].map(posiciones_x)

# ---------- H) Renombrar para consistencia ----------
data = df.rename(columns={'ano': 'time', 'hectareas': 'value'})

# ---------- I) Crear gráficos ----------
# Selector de especie
species_selector = alt.selection_point(fields=['id'])

# Selector de año - usando un slider
year_slider = alt.binding_range(
    name='Selecciona el año:',
    min=int(data['time'].min()),
    max=int(data['time'].max()),
    step=1
)
year_selection = alt.param(value=int(data['time'].max()), bind=year_slider)

# Color condicional
color = alt.condition(
    species_selector,
    alt.Color('id:O', legend=None),
    alt.value('lightgray')
)

# Gráfico de burbujas - para el año seleccionado
bubbles = alt.Chart(data).mark_circle(filled=True, opacity=0.8, size=200).encode(
    x=alt.X('x_pos:O', 
            axis=alt.Axis(title='Especies', labels=False, ticks=False),
            scale=alt.Scale(padding=0.5)),
    y=alt.value(150),  # Todas en la misma línea horizontal
    size=alt.Size('value:Q',
                 title='Hectáreas',
                 scale=alt.Scale(range=[100, 2000]),  # Ajustar según tus datos
                 legend=alt.Legend(orient='bottom')),
    color=color,
    tooltip=[
        'especie:N', 
        'time:O',
        alt.Tooltip('value:Q', title='Hectáreas', format=',.0f')
    ]
).properties(
    width=600,
    height=200,
    title='Hectáreas por Especie - Año Seleccionado'
).add_params(
    species_selector, year_selection
).transform_filter(
    alt.datum.time == year_selection  # Filtrar por el año seleccionado
)

# Gráfico de series temporales
line = alt.Chart(data).mark_line(point=True, strokeWidth=3).encode(
    x=alt.X('time:O', title='Año'),
    y=alt.Y('value:Q', title='Hectáreas', scale=alt.Scale(zero=False)),
    color=alt.Color('id:O', legend=None),
    tooltip=['especie:N','time:O','value:Q']
).transform_filter(
    species_selector
).properties(
    width=600,
    height=300,
    title='Evolución Temporal de la Especie Seleccionada'
)

# Gráfico de barras para comparar especies en el año seleccionado
bars = alt.Chart(data).mark_bar().encode(
    x=alt.X('especie:N', title='Especies', sort='-y'),
    y=alt.Y('value:Q', title='Hectáreas'),
    color=color,
    tooltip=['especie:N', alt.Tooltip('value:Q', title='Hectáreas', format=',.0f')]
).properties(
    width=600,
    height=300,
    title='Comparación de Especies - Año Seleccionado'
).add_params(
    species_selector, year_selection
).transform_filter(
    alt.datum.time == year_selection  # Filtrar por el año seleccionado
)

# Mostrar los gráficos en vertical
chart = alt.vconcat(
    bubbles,
    line,
    bars
).configure_title(fontSize=14, anchor='start')

chart

Columnas originales: ['especie', 'año', 'hectareas', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26']
Primeras filas (raw):
 especie    año  hectareas  Unnamed: 3  Unnamed: 4  Unnamed: 5  Unnamed: 6  Unnamed: 7  Unnamed: 8  Unnamed: 9  Unnamed: 10  Unnamed: 11  Unnamed: 12  Unnamed: 13  Unnamed: 14  Unnamed: 15  Unnamed: 16  Unnamed: 17  Unnamed: 18  Unnamed: 19  Unnamed: 20  Unnamed: 21  Unnamed: 22  Unnamed: 23  Unnamed: 24  Unnamed: 25  Unnamed: 26
Almendro 2024.0    8324.11         NaN         NaN         NaN         NaN         NaN         NaN         NaN          NaN          NaN          NaN          NaN          NaN          NaN          NaN          NaN          NaN          NaN

alt.VConcatChart(...)

In [43]:
# ---------- I) Crear gráficos ----------
# Selector de especie
species_selector = alt.selection_point(fields=['id'])

# Selector de año - usando un slider
year_slider = alt.binding_range(
    name='Selecciona el año:',
    min=int(data['time'].min()),
    max=int(data['time'].max()),
    step=1
)
year_selection = alt.param(value=int(data['time'].max()), bind=year_slider)

# Paleta de verdes pistacho personalizada
pistacho_colors = [
    '#8A9A5B',  # Verde pistacho base
    '#9CAF78',  # Pistacho claro
    '#7A8C4F',  # Pistacho medio
    '#6B7D45',  # Pistacho oscuro
    '#5C6E3B',  # Pistacho más oscuro
    '#A3B58A',  # Pistacho muy claro
    '#4A5A2F',  # Pistacho intenso
    '#B8C9A3'   # Pistacho pastel
]

# Color condicional con verdes pistacho
color = alt.condition(
    species_selector,
    alt.Color('id:O', scale=alt.Scale(range=pistacho_colors), legend=None),
    alt.value('#E0E0E0')  # Gris claro para no seleccionados
)

# Gráfico de burbujas - para el año seleccionado
bubbles = alt.Chart(data).mark_circle(filled=True, opacity=0.8, size=200).encode(
    x=alt.X('x_pos:O', 
            axis=alt.Axis(title='Especies', labels=False, ticks=False),
            scale=alt.Scale(padding=0.5)),
    y=alt.value(150),  # Todas en la misma línea horizontal
    size=alt.Size('value:Q',
                 title='Hectáreas',
                 scale=alt.Scale(range=[100, 2000]),
                 legend=alt.Legend(orient='bottom')),
    color=color,
    tooltip=[
        'especie:N', 
        'time:O',
        alt.Tooltip('value:Q', title='Hectáreas', format=',.0f')
    ]
).properties(
    width=600,
    height=200,
    title='Hectáreas por Especie - Año Seleccionado'
).add_params(
    species_selector, year_selection
).transform_filter(
    alt.datum.time == year_selection
)

# Gráfico de series temporales
line = alt.Chart(data).mark_line(point=True, strokeWidth=3).encode(
    x=alt.X('time:O', title='Año'),
    y=alt.Y('value:Q', title='Hectáreas', scale=alt.Scale(zero=False)),
    color=alt.Color('id:O', scale=alt.Scale(range=pistacho_colors), legend=None),
    tooltip=['especie:N','time:O','value:Q']
).transform_filter(
    species_selector
).properties(
    width=600,
    height=300,
    title='Evolución Temporal de la Especie Seleccionada'
)

# Gráfico de barras para comparar especies en el año seleccionado
bars = alt.Chart(data).mark_bar().encode(
    x=alt.X('especie:N', title='Especies', sort='-y'),
    y=alt.Y('value:Q', title='Hectáreas'),
    color=color,
    tooltip=['especie:N', alt.Tooltip('value:Q', title='Hectáreas', format=',.0f')]
).properties(
    width=600,
    height=300,
    title='Comparación de Especies - Año Seleccionado'
).add_params(
    species_selector, year_selection
).transform_filter(
    alt.datum.time == year_selection
)

# Mostrar los gráficos en vertical con tema verde
chart = alt.vconcat(
    bubbles,
    line,
    bars
).configure_title(
    fontSize=14, 
    anchor='start'
).configure_view(
    strokeWidth=0
).configure_axis(
    labelColor='#4A5A2F',
    titleColor='#4A5A2F'
).configure_header(
    labelColor='#4A5A2F',
    titleColor='#4A5A2F'
)

chart

alt.VConcatChart(...)